# Parsing

## CSV Parsing

In [ ]:
# importing the Python standard library 'csv'
import csv

### Reading CSV

In [ ]:
with open("files/parasite_data.csv", 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter=',', dialect='excel')
    for index, row in enumerate(reader):
        if index == 0:
            headers = row
            continue
        print ', '.join(row)
        
    print headers

**DictReader**

In [ ]:
with open('files/parasite_data.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', dialect='excel')
    for row in reader:
        print row['ShannonDiversity'], 
        print row['Virulence'],
        print row['Replicate']

### Writing CSV

In [ ]:
with open("files/example_output.csv", 'w') as csv_file:
    writer = csv.writer(csv_file, delimiter=',', dialect='excel')
    writer.writerow(['Spam'] * 5 + ['Baked Beans'])
    writer.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam']) 

In [ ]:
# %load files/example_output.csv
Spam,Spam,Spam,Spam,Spam,Baked Beans
Spam,Lovely Spam,Wonderful Spam


**Dictwriter**

In [ ]:
with open("files/example_output.csv", 'w') as csv_file:
    headers = ['first_name', 'last_name']
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()
    writer.writerow({'first_name': 'Baked', 'last_name': 'Beans'})
    writer.writerow({'first_name': 'Lovely', 'last_name': 'Spam'})
    writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})

In [ ]:
# %load files/example_output.csv
first_name,last_name
Baked,Beans
Lovely,Spam
Wonderful,Spam


## JSON Parsing

In [ ]:
# importing the Python standard library 'json'
import json

### Importing JSON to Dict { }

**From string**

In [ ]:
# Example string
json_string = '{"first_name": "Guido", "last_name":"Rossum"}'
json_string
type(json_string)

In [ ]:
parsed_json = json.loads(json_string)
parsed_json
type(parsed_json)

**From file**

In [ ]:
# %load files/example.json
{
    "glossary": {
        "title": "example glossary",
    "GlossDiv": {
            "title": "S",
      "GlossList": {
                "GlossEntry": {
                    "ID": "SGML",
          "SortAs": "SGML",
          "GlossTerm": "Standard Generalized Markup Language",
          "Acronym": "SGML",
          "Abbrev": "ISO 8879:1986",
          "GlossDef": {
                        "para": "A meta-markup language, used to create markup languages such as DocBook.",
            "GlossSeeAlso": ["GML", "XML"]
                    },
          "GlossSee": "markup"
                }
            }
        }
    }
}

In [ ]:
# with open('files/example.json', 'r') as f:
#     parsed_json = json.load(f)
    
parsed_json
parsed_json['glossary']['GlossDiv']['GlossList']['GlossEntry']['Abbrev']

### Exporting Dict { } to JSON

In [ ]:
# Example dict
d = { 'first_name': 'Guido',
      'second_name': 'Rossum',
      'titles': ['BDFL', 'Developer'],
    }
d

In [ ]:
json_string = json.dumps(d)
json_string

In [ ]:
with open('files/example_output.json', 'w') as f:
    f.write(json_string)

In [ ]:
# %load files/example_output.json
{"first_name": "Guido", "titles": ["BDFL", "Developer"], "second_name": "Rossum"}

## XML Parsing

In [ ]:
# importing the Python standard library 'xml'
import xml.etree.ElementTree as ET

### Importing XML to ElementTree

**From string**

In [ ]:
# Example string
xml_string = """<catalog id="my_catalog">
   <book id="bk101">
      <author>Gambardella, Matthew</author>
      <title>XML Developer's Guide</title>
      <genre>Computer</genre>
   </book>
</catalog>"""
xml_string

In [ ]:
root = ET.fromstring(xml_string)
print root.tag
print root.attrib

**From file**

In [ ]:
# %load files/example.xml
<?xml version="1.0"?>
<catalog>
   <book id="bk101">
      <author>Gambardella, Matthew</author>
      <title>XML Developer's Guide</title>
      <genre>Computer</genre>
      <price>44.95</price>
      <publish_date>2000-10-01</publish_date>
      <description>An in-depth look at creating applications
      with XML.</description>
   </book>
   <book id="bk103">
      <author>Corets, Eva</author>
      <title>Maeve Ascendant</title>
      <genre>Fantasy</genre>
      <price>5.95</price>
      <publish_date>2000-11-17</publish_date>
      <description>After the collapse of a nanotechnology
      society in England, the young survivors lay the
      foundation for a new society.</description>
   </book>
   <book id="bk109">
      <author>Kress, Peter</author>
      <title>Paradox Lost</title>
      <genre>Science Fiction</genre>
      <price>6.95</price>
      <publish_date>2000-11-02</publish_date>
      <description>After an inadvertant trip through a Heisenberg
      Uncertainty Device, James Salway discovers the problems
      of being quantum.</description>
   </book>
</catalog>

In [ ]:
tree = ET.parse('files/example.xml')
root = tree.getroot()
root.tag
root.text

### Exporting ElementTree to XML

In [ ]:
tree.write("files/example_output.xml")

In [ ]:
# %load files/example_output.xml
<catalog>
   <book id="bk101">
      <author>Gambardella, Matthew</author>
      <title>XML Developer's Guide</title>
      <genre>Computer</genre>
      <price>44.95</price>
      <publish_date>2000-10-01</publish_date>
      <description>An in-depth look at creating applications
      with XML.</description>
   </book>
   <book id="bk103">
      <author>Corets, Eva</author>
      <title>Maeve Ascendant</title>
      <genre>Fantasy</genre>
      <price>5.95</price>
      <publish_date>2000-11-17</publish_date>
      <description>After the collapse of a nanotechnology
      society in England, the young survivors lay the
      foundation for a new society.</description>
   </book>
   <book id="bk109">
      <author>Kress, Peter</author>
      <title>Paradox Lost</title>
      <genre>Science Fiction</genre>
      <price>6.95</price>
      <publish_date>2000-11-02</publish_date>
      <description>After an inadvertant trip through a Heisenberg
      Uncertainty Device, James Salway discovers the problems
      of being quantum.</description>
   </book>
</catalog>

### Getting items

In [ ]:
# Each XML element contains a tag, an attribute (optional), a text and a list of childs
# XML Element: <tag attr:attr_value> text </tag>

print root.tag            # tag    : name of XML element
print root.attrib         # attrib : attribute of XML element
print root.text           # text   : content of XML element
for child in root: # loop through all subchildrens
    print "\t", child.tag, child.attrib, child.text
    for subchild in child:
        print "\t\t", subchild.tag, subchild.attrib, subchild.text
    print

In [ ]:
children = list(root)     #Get children

for child in children:
    print child.tag
    print child.attrib
    print child.text
    for subchild in child:
        print child.tag
        print child.attrib
        print child.text
    
print children
print root

In [ ]:
# iter('element_tag') - search all subtrees
for title in root.iter('title'): 
    print title.text

In [ ]:
# findall('element_tag') - get direct childs of parent
for book in root.findall('book'):
    # Get items
    book_id = book.get('id')          # Get attribute 'id' of 'book' element
    title = book.find('title')        # Get first child named 'title'
    author = book.find('author')
    genre = book.find('genre')
    price = book.find('price')
    pdate = book.find('publish_date')
    descr = book.find('description')
    
    # Print items
    print "ID ", book_id
    print title.tag, title.attrib, title.text
    print author.tag, author.attrib, author.text
    print genre.tag, genre.attrib, genre.text
    print price.tag, price.attrib, price.text
    print pdate.tag, pdate.text
    print descr.tag, descr.text

In [ ]:
# find('element_tag') - finds first child with tag
for book_content in root.find('book'):
    print book_content

### Modifying items

In [ ]:
# set('attribute', 'attribute_value')
for price in root.iter('price'):
    price.text = str(float(price.text) + 1)
    price.set('increased', 'yes')
    print price.text, price.attrib

In [ ]:
for book in root.iter('book'):
    book.set('id', book.attrib['id'] + '0')
    print book.attrib

for price in root.iter('price'):
    price.text = str(float(price.text) + 1)
    price.attrib['increased'] = 'no'
    price.set('book', )
    
ET.dump(tree)
tree.write('example_output.xml')

In [ ]:
# Create new element
new_book = ET.Element('book', {'id': 'bk113'})
new_book.text = "\n"
                      
# Create sub elements
author = ET.SubElement(new_book, 'author')
title = ET.SubElement(new_book, 'title')
genre = ET.SubElement(new_book, 'genre')
price = ET.SubElement(new_book, 'price')
pdate = ET.SubElement(new_book, 'publish_date')
descr = ET.SubElement(new_book, 'description')

# Populate sub elements
author.text = "J.K Rowlings"
title.text = "Harry Potter and the Sorcerer's Stone"
genre.text = "Fantasy"
price.text = "31.50"
pdate.text = "2001-10-16"
descr.text = "A very nice fantasy book."

# Add element to existing tree
root.append(new_book)

ET.dump(root)

In [ ]:
# A ten times nicer way of doing it (focus on what matters)

def create_new_ET(element_name, attributes={}, elements={}):
    book = ET.Element(element_name, attributes)
    for key in elements:
        new_elem = ET.SubElement(book, key)
        new_elem.text = elements[key]
    return book

if __name__ == '__main__':
    # Create new element
    name     = 'book'
    attr     = {'id': "bk114"}
    elements = {'author': "J.K Rowlings",
                'title': "Harry Potter and the Chamber of Secrets",
                'genre': "Fantasy",
                'price': "35.50",
                'publish_date': "2002-10-15",
                'description': "The second volume of a very nice fantasy book."}
    new_book = create_new_ET(name, attr, elements)

    # Append to existing tree
    root.append(new_book)

ET.dump(root)

## Bonus: Pretty Print and Conversion

### Pretty Print

In [ ]:
def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [ ]:
indent(root)
tree.write("files/example_output.xml")
ET.dump(tree)

In [ ]:
# %load files/example_output.xml
<catalog>
  <book id="bk101">
    <author>Gambardella, Matthew</author>
    <title>XML Developer's Guide</title>
    <genre>Computer</genre>
    <price increased="yes">45.95</price>
    <publish_date>2000-10-01</publish_date>
    <description>An in-depth look at creating applications
      with XML.</description>
  </book>
  <book id="bk103">
    <author>Corets, Eva</author>
    <title>Maeve Ascendant</title>
    <genre>Fantasy</genre>
    <price increased="yes">6.95</price>
    <publish_date>2000-11-17</publish_date>
    <description>After the collapse of a nanotechnology
      society in England, the young survivors lay the
      foundation for a new society.</description>
  </book>
  <book id="bk109">
    <author>Kress, Peter</author>
    <title>Paradox Lost</title>
    <genre>Science Fiction</genre>
    <price increased="yes">7.95</price>
    <publish_date>2000-11-02</publish_date>
    <description>After an inadvertant trip through a Heisenberg
      Uncertainty Device, James Salway discovers the problems
      of being quantum.</description>
  </book>
  <book id="bk113">
    <author>J.K Rowlings</author>
    <title>Harry Potter and the Sorcerer's Stone</title>
    <genre>Fantasy</genre>
    <price>31.50</price>
    <publish_date>2001-10-16</publish_date>
    <description>A very nice fantasy book.</description>
  </book>
  <book id="bk114">
    <description>The second volume of a very nice fantasy book.</description>
    <title>Harry Potter and the Chamber of Secrets</title>
    <price>35.50</price>
    <author>J.K Rowlings</author>
    <publish_date>2002-10-15</publish_date>
    <genre>Fantasy</genre>
  </book>
</catalog>


### Etree to Dict { }

In [ ]:
from collections import defaultdict

def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.iteritems():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.iteritems()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.iteritems())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [ ]:
d = etree_to_dict(root)
d

### Dict { } to Etree

In [ ]:
def dict_to_etree(d):
    def _to_etree(d, root):
        if not d:
            pass
        elif isinstance(d, basestring):
            root.text = d
        elif isinstance(d, dict):
            for k,v in d.items():
                assert isinstance(k, basestring)
                if k.startswith('#'):
                    assert k == '#text' and isinstance(v, basestring)
                    root.text = v
                elif k.startswith('@'):
                    assert isinstance(v, basestring)
                    root.set(k[1:], v)
                elif isinstance(v, list):
                    for e in v:
                        _to_etree(e, ET.SubElement(root, k))
                else:
                    _to_etree(v, ET.SubElement(root, k))
        else: assert d == 'invalid type', (type(d), d)
    assert isinstance(d, dict) and len(d) == 1
    tag, body = next(iter(d.items()))
    node = ET.Element(tag)
    _to_etree(body, node)
    return node

In [ ]:
t = dict_to_etree(d)
indent(t)
ET.dump(t)

## Exercise: Analyze XML file
### Problem
**Objectives:** 
- Extract Maven plugins information from ```base-corporate-pom```.
- Add a new Maven plugin to ```base-corporate-pom```.
- Write Maven plugins information to a new file ```files/pom_maven_plugins.xml```

**Information:**

- ```base-corporate-pom``` pom.xml is located in ```files/pom.xml```


- XML plugin structure:
```
<project>
    <!-- Plugin version -->
    <properties>
        <maven-surefire-plugin.version>2.12.4</maven-surefire-plugin.version>
    </properties>
    <!-- Plugin info -->
    <build>
        <pluginManagement>
            <plugins>
                <plugin>
                    <groupId>org.apache.maven.plugin</groupId>
                    <artifactId>maven-surefire-plugin</version>
                    <version>$(maven-surefire-plugin.version)</version>
                 </plugin>
            </plugins>
         </pluginManagement>
    </build>
</project>
```
- Some plugins don't have a ```<version>``` or a ```<groupId>``` tag. Print "N/A" when there is no tag.
- All plugins have an ```<artifactId>``` tag.

**Desired output:**

maven-surefire

    groupId: org.apache.maven.plugins
    artifactId: maven-surefire-plugin
    version: 2.12.4

### Solution 1

In [ ]:
# %load solutions/xml_solution.py
import xml.etree.ElementTree as ET

root = ET.parse('files/pom.xml').getroot()

# Variables
plugin_list = []
plugin_dict = {}
base = "{http://maven.apache.org/POM/4.0.0}"

# Find 'build' and 'properties' XML tags in root
build = root.find(base + 'build')
properties = root.find(base + 'properties')

# Find 'pluginManagement' XML tag in 'build'
pluginManagement = build.find(base + 'pluginManagement')

# Find 'plugins' XML tag in 'pluginManagement'
plugins = pluginManagement.find(base + 'plugins')

# Populate plugin_list
plugin_list = list(plugins)

# Loop through plugins (of class ElementTree)
for plugin in plugin_list:

        # Get  plugin information
        artifactId_str = plugin.find(base + 'artifactId').text
        name = artifactId_str.replace('-plugin', '')

        # Not all plugins have a groupId !
        try:
            groupId_str = plugin.find(base + 'groupId').text
        except:
            groupId_str = "N/A"

        # Not all plugins have a version !
        try:
            version_str = plugin.find(base + 'version').text
            version_str = version_str[2:-1] # strip '$', '{', and '}' from version
            version_str = properties.find(base + version_str).text
        except:
            version_str = "N/A"

        # Print plugin information
        print artifactId_str.replace('-plugin', '')
        print "\t" + "groupId: " + groupId_str
        print "\t" + "artifactId: " + artifactId_str
        print "\t" + "version: " + version_str

        # Add plugin to dictionary
        plugin_dict[name] = {'artifactId': artifactId_str,
                             'groupId': groupId_str,
                             'version': version_str
                             }

        # Write dict to file:
        with open('files/pom_maven_plugins_output.xml', 'w') as f:
            for pname, sub in plugin_dict.iteritems():
                f.write(pname + "\n")
                for k, v in sub.iteritems():
                    f.write("\t" + k + ": " + v + "\n")

### Solution 2 (improved)

In [ ]:
# %load solutions/xml_solution2.py
import xml.etree.ElementTree as ET
import re

def concat(list_tags):
    # Small function to concatenate the base with the tag name for a list of names.
    # Return a tuple containing the new names.
    t = tuple()
    for tag in list_tags:
        t += (base + tag,)
    return t

def build_path(list_, base = ''):
    # Create a path from a list of subpaths.
    # If a base is defined, append the based before each path element.
    path = "."
    for a in list_:
        if base:
            path += "/" + base + a
            continue
        path += "/" + a
    return path

def get_plugin_info(plugin):
    # Get concatenated names
    names = ['artifactId', 'groupId', 'version']
    aId, gId, v = concat(names)

    # Get elements from plugin ET
    artifactId = plugin.find(aId)
    groupId = plugin.find(gId)
    version = plugin.find(v)

    # Set output values

    try:
        aId_str = artifactId.text
    except Exception as e:
        raise(e)

    try:
        gId_str = groupId.text
    except:
        gId_str = "N/A"

    try:
        version_str = version.text # strip '$', '{', and '}' from version.text
        try:
            float(version_str)
            version_str = str(version_str)
        except ValueError:
            version_str = version_str[2:-1]
            version_str = properties.find(base + version_str).text
    except:
        version_str = "N/A"

    # Return plugin information
    name_str = aId_str.replace('-plugin','')
    return (aId_str, gId_str, version_str, name_str)

def format_plugin_info(plugin):
    artifactId, groupId, version, name = get_plugin_info(plugin)
    to_print = name + "\n" + \
    "\t" + "groupId: " + groupId + "\n" + \
    "\t" + "artifactId: " + artifactId + "\n" + \
    "\t" + "version: " + version
    return to_print

def print_plugin(plugin):
    print format_plugin_info(plugin)

def write_plugin_to_file(plugin, filepath):
    to_print = format_plugin_info(plugin)
    with open(filepath, 'a') as f:
        f.write(to_print)
        f.write("\n")

# main function
if __name__ == '__main__':
    root = ET.parse('files/pom.xml').getroot()
    base = "{http://maven.apache.org/POM/4.0.0}"

    # Get plugins Element Tree element
    plugins_path    = build_path(['build', 'pluginManagement', 'plugins'], base=base)
    plugins = root.find(plugins_path)

    # Get plugins info and print plugins
    for p in plugins.findall(base + 'plugin'):
        print_plugin(p)
        write_plugin_to_file(p, 'files/pom_maven_plugins_output.xml')

In [ ]:
def get_elements(search_string, root, base=''):
    try:
        search_string = base + search_string
        return root.iter(search_string)
    except:
        print "Your tag doesn't exist."
        
if __name__ == '__main__':
    base = "{http://maven.apache.org/POM/4.0.0}"
    root = ET.parse('files/pom.xml').getroot()
    plugins = get_elements('plugin', root, base="{http://maven.apache.org/POM/4.0.0}")
    for plugin in plugins:
        print plugin.text
        
        try:
            groupId = plugin.find(base + 'groupId').text
        except AttributeError:
            groupId = "N/A"
            
        artifactId = plugin.find(base + 'artifactId').text
        
        try:
            version = plugin.find(base + 'version').text
        except AttributeError:
            version ="N/A"
                
        try:
            version_new = float(version)
        except:
            try:
                version_new = version.replace('$', '').replace('{', '').replace('}', '')
                string = './'
                string += base + 'properties' + '/' + base + version_new
                version_new = root.find(string).text
            except:
                version_new = "N/A"

        print artifactId
        print groupId
        print version_new

### Solution 3 (OOP Approach)

In [ ]:
# %load solutions/xml_solution3.py
import xml.etree.ElementTree as ET
import re

class Plugin(object):
    """ A plugin has a name, an artifactId, a groupId and a version."""
    def __init__(self, plugin, base):
        self.base = base
        self.__get_plugin_info(plugin)

    def __repr__(self):
        to_print = self.name + "\n" + \
        "\t" + "groupId: " + self.groupId + "\n" + \
        "\t" + "artifactId: " + self.artifactId + "\n" + \
        "\t" + "version: " + self.version
        return to_print

    # PRIVATE
    def __concat(self, list_tags):
        """Small function to concatenate the base with the tag name for a list of names.
        Return a tuple containing the new names."""
        t = tuple()
        for tag in list_tags:
            t += (self.base + tag,)
        return t

    def __get_plugin_info(self, plugin):
        """ Takes a plugin of class ElementTree and populates this Plugin object.
        Fields are: name, artifactId, groupid, version"""
        # Get concatenated names
        names = ['artifactId', 'groupId', 'version']
        aId, gId, v = self.__concat(names)

        # Get elements from plugin ET
        artifactId = plugin.find(aId)
        groupId = plugin.find(gId)
        version = plugin.find(v)

        # Set output values
        try:
            aId_str = artifactId.text
        except Exception as e:
            raise(e)

        try:
            gId_str = groupId.text
        except:
            gId_str = "N/A"

        try:
            version_str = version.text # strip '$', '{', and '}' from version.text
            try:
                float(version_str)
                version_str = str(version_str)
            except ValueError:
                version_str = version_str[2:-1]
                version_str = properties.find(base + version_str).text
        except:
            version_str = "N/A"

        name_str = aId_str.replace('-plugin','')

        # Populate Plugin object
        self.artifactId = aId_str
        self.groupId = gId_str
        self.version = version_str
        self.name = name_str

class POMPluginExtractor(object):
    """This class serves as extractor of plugins from any POM file."""
    def __init__(self, filepath, base):
        self.filepath = filepath
        self.base = base
        self.plugins = []
        self.__get_root()
        self.__get_plugins()

    def print_plugins(self):
        for p in self.plugins:
            print p

    def save_plugins(self, filepath):
        with open(filepath, 'w') as f:
            for p in self.plugins:
                print >>f, p
                print >>f

        print "Plugins saved to %s" % filepath

    # PRIVATE
    def __get_root(self):
        self.root = ET.parse(self.filepath).getroot()

    def __get_plugins(self):
        plugins_path = self.__build_path(['build', 'pluginManagement', 'plugins'], base=self.base)
        plugins = self.root.find(plugins_path)
        for p in plugins.findall(self.base + 'plugin'):
            self.plugins.append(Plugin(p, self.base))

    def __build_path(self, list_, base = ''):
        path = "."
        for a in list_:
            if base:
                path += "/" + base + a
                continue
            path += "/" + a
        return path

# main function
if __name__ == '__main__':
    inputFile = 'files/pom.xml'
    outputFile = 'files/pom_maven_plugins_output.xml'
    base = "{http://maven.apache.org/POM/4.0.0}"

    analyzer = POMPluginExtractor(inputFile, base)
    analyzer.save_plugins(outputFile)
    analyzer.print_plugins()

# REST API (Django)
**The following section follows the excellent Django tutorial at https://ultimatedjango.com/blog/how-to-consume-rest-apis-with-django-python-reques/**

**In this tutorial you will learn how to use Django, Django-Rest-Framework, and the Python requests package to consume data from an external REST API. We will use the external API from [embed.ly](http://embed.ly/) for this tutorial.**

* **Modules required**: ```django```, ```djangorestframework```, ```requests```
* **Installation**: 
    * ```pip install django``` - http://www.scipy.org/scipylib/download.html
    * ```pip install djangorestframework```
    * ```pip install requests```

## Django Setup

**Create a new project & app**

In [ ]:
%%bash

# create our test directory
cd ~
mkdir restest
cd restest

# start a new project 'restembed'
django-admin startproject restembed .

# open the project directory and create a new app 'core'
cd restembed
django-admin startapp core

**Make a few additions to ```settings.py```**

* Open ```restembed/settings.py```.
* Update ```settings.py``` as shown in the following (highlighted)

<img src="files/django_pic1.png" align="left">
<img src="files/django_pic2.png" align="left">
<img src="files/django_pic3.png" align="left">

**Commit the database migrations**

In [ ]:
%%bash
# go to project folder
cd ~/restest

# commit the migrations
python manage.py migrate

# create a super user (will not work from ipython)
python manage.py createsuperuser

## Create a model
**At the core of the application is a simple model. It is based on the data set returned from the Embed.ly web service. **

* Open ```restembed/core/models.py```.
* Write the following code in it.

In [ ]:
%%writefile ~/restest/restembed/core/models.py

from __future__ import unicode_literals
from django.db import models

class SavedEmbeds(models.Model):
    type = models.CharField(max_length=15)
    provider_url = models.URLField()
    provider_name = models.CharField(max_length=100)
    title = models.CharField(max_length=100)
    description = models.TextField()
    html = models.TextField()
    width = models.IntegerField()
    height = models.IntegerField()
    thumbnail_url = models.URLField()
    thumbnail_width = models.IntegerField()
    thumbnail_height = models.IntegerField()
    author_url = models.URLField()
    author_name = models.CharField(max_length=100)
    version = models.DecimalField(max_digits=4, decimal_places=2)

## Create a serializer
**The application will make a GET request to the Embed.ly web service. Embed.ly will return a JSON response. We need to save that response to the database. A serializer offers us a mechanism that can perform this easily.**

* In ```restembed/core/```, create a file named ```serializer.py```.
* Write the following code in it

In [ ]:
%%writefile ~/restest/restembed/core/serializer.py

from rest_framework import serializers
from .models import SavedEmbeds

class EmbedSerializer(serializers.ModelSerializer):
    class Meta:
        model = SavedEmbeds

## Create a view
**A view will process the incoming requests.**

* Open ```restembed/core/views.py```.
* Write the following code in it.

In [ ]:
%%writefile ~/restest/restembed/core/views.py

from django.shortcuts import render
from django.conf import settings

import requests


from .forms import SubmitEmbed
from .serializer import EmbedSerializer


def save_embed(request):

    if request.method == "POST":
        form = SubmitEmbed(request.POST)
        if form.is_valid():
            url = form.cleaned_data['url']
            r = requests.get('http://api.embed.ly/1/oembed?key=' + settings.EMBEDLY_KEY + '&url=' + url)
            json = r.json()
            serializer = EmbedSerializer(data=json)
            if serializer.is_valid():
                embed = serializer.save()
                return render(request, 'embeds.html', {'embed': embed})
    else:
        form = SubmitEmbed()

    return render(request, 'index.html', {'form': form})

## Create a Form
**We’ll need a simple form to collect the URL for the end user. This form will have one field. That field will allow a user to enter a URL that they want saved to the database.**

* Open ```restembed/core/forms.py```.
* Write the following code in it.

In [ ]:
%%writefile ~/restest/restembed/core/forms.py

from django import forms

class SubmitEmbed(forms.Form):
    url = forms.URLField()

## Create the URLs
**We'll need one additional URL configuration for this tutorial. This will allow users to open the root of the domain and see the form that will allow them to submit a URL.**

* Open ```restembed/urls.py```.
* Write the following code in it.

In [ ]:
%%writefile ~/restest/restembed/urls.py
from django.conf.urls import patterns, include, url
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^', include('restembed.core.urls')),
]

* Create ```restembed/core/urls.py```.
* Write the following code in it

In [ ]:
%%writefile ~/restest/restembed/core/urls.py

from django.conf.urls import include, url
from . import views

urlpatterns = [
    url(r'^$', views.save_embed, name='save_embed'),
]

## Create the Templates
**We'll need three templates.**

* Go to the ```restembed/``` directory and create a folder there named ```templates```.
* Open ```restembed/templates/``` folder and create the following three templates files.

In [ ]:
%%bash

cd ~/restest/restembed
mkdir templates

**```templates/base.html```**

In [ ]:
%%writefile ~/restest/restembed/templates/base.html

<html>
    <body>
        <div>
            {% block content %}{% endblock %}
        </div>
    </body>
</html>

**```templates/index.html```**

In [ ]:
%%writefile ~/restest/restembed/templates/index.html

{% extends 'base.html' %}

{% block content %}

    <form action="/" method="POST">
        {% csrf_token %}
        {{ form }}
        <input type="submit" value="Save"/>
    </form>


{% endblock %}

**```templates/embeds.html```**

In [ ]:
%%writefile ~/restest/restembed/templates/embeds.html

{% extends 'base.html' %}

{% block content %}

    <ul>
        <li>{{ embed.title }}</li>
        <li>{{ embed.description }}</li>
    </ul>
    <img src="{{ embed.thumbnail_url }}" />

{% endblock %}

## Setup Admin
**In order to ensure the code is working properly, you can activate the admin console to look at records.**

* Open ```restembed/core/admin.py```.
* Write the following in it.

In [ ]:
%%writefile ~/restest/restembed/core/admin.py

from django.contrib import admin

from .models import SavedEmbeds

admin.site.register(SavedEmbeds)

## Test the Application
**The last step is to test the application.**

In [ ]:
%%bash

cd ~/restest
python manage.py makemigrations
python manage.py migrate

python manage.py runserver

# SQL Data Access (MySQL)
* **Module required**: ```mysql-connector```
* **Installation**: ```pip install --allow-external mysql-connector-python mysql-connector-python``` - https://geert.vanderkelen.org/2014/install-mysqlcpy-using-pip/

In [ ]:
import mysql.connector

## Connecting to MySQL DB

In [ ]:
# requires a local MySQL server running on localhost
cnx = mysql.connector.connect(user='root', password='password',
                              host='localhost')
cnx.close()
cnx

**Lots of connection arguments**

In [ ]:
# LOCAL
config = {
  'user': 'root',
  'password': 'wrong_password',
  'host': 'localhost',
  'database': 'tutorial',
  'raise_on_warnings': True,
}

cnx = mysql.connector.connect(**config)
cnx.close()
cnx

**Handling connection errors**

In [ ]:
from mysql.connector import errorcode

config = {
  'user': 'root',
  'password': 'wrong_password',
  'host': 'localhost',
  'raise_on_warnings': True,
}

# Exception handling code
try:
    cnx = mysql.connector.connect(**config)
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print "[ERROR] Something is wrong with you user name or password"
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print "[ERROR] Database does not exist."
    else:
        print err
else:
    cnx.close()

**Executing SQL statements**

***Note: All DDL (Data Definition Language) statements are executed using a handle structure known as 'cursor'.***

In [ ]:
# A basic example for executing SQL queries

from mysql.connector import MySQLConnection

config = {
  'user': 'root',
  'password': 'password',
  'host': 'localhost',
  'raise_on_warnings': True,
}

def exec_query(cursor, query, query_args=None):
    try:
        if query_args is not None:
            print "[INFO]  Executing SQL: \"%s\"..." % (query % query_args),
            cursor.execute(query, query_args)
        else:
            print "[INFO]  Executing SQL: \"%s\"..." % query,
            cursor.execute(query)
        print " SUCCESS"
    except mysql.connector.Error as err:
        print
        print "[ERROR] " + err.msg
        exit(1)
        
if __name__ == '__main__':
    # SQL statements
    query_1 = "CREATE DATABASE test"
    query_2 = "CREATE TABLE people (first_name VARCHAR(25) PRIMARY KEY, last_name VARCHAR(25))"
    query_3 = "INSERT people(first_name, last_name) VALUES (%s, %s)"
    query_3_args = ("Olivier", "Cervello")
    query_4 = "SELECT * FROM people"
    query_4 = "DROP TABLE people"
    query_5 = "DROP DATABASE test"
    
    #connect to MySQL server
    cnx = MySQLConnection(**config)
    cursor = cnx.cursor()
    
    #create database 'test'
    exec_query(cursor, query_1)
    
    # connect to database 'test'
    config['database'] = 'test'
    cnx.connect(**config)
    
    # create table 'people' and insert a value
    exec_query(cursor, query_2)
    exec_query(cursor, query_3, query_3_args)
    
    # Commit changes (query_3)
    cnx.commit()
    
    # delete databases
#     exec_query(cursor, query_4)
#     exec_query(cursor, query_5)

    # close connection
    cnx.close()

## Creating / Deleting a database

In [ ]:
from mysql.connector import MySQLConnection
from mysql.connector import errorcode

config = {
  'user': 'root',
  'password': 'password',
  'host': 'localhost',
  'raise_on_warnings': True,
}

def create_database(cursor, db_name):
    query = "CREATE DATABASE %s" % db_name
    exec_query(cursor, query)
        
def get_database(cnx, db_name):
    cursor = cnx.cursor()
    print "[INFO]  Fetching database %s ..." % db_name
    try:
        cnx.database = db_name    
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_BAD_DB_ERROR:
            print "[ERROR] Database %s does not exist! Creating database..." % db_name
            create_database(cursor, db_name)
            cnx.database = db_name
        else:
            print(err)
            exit(1)
    print "[INFO]  Database %s selected." % db_name

def delete_database(cursor, db_name):
    query = "DROP DATABASE %s" % db_name
    exec_query(cursor, query)

if __name__ == '__main__':
    cnx = MySQLConnection(**config)
    cursor = cnx.cursor()
    get_database(cnx, 'new_db')
    delete_database(cursor, 'new_db')

## Creating / Deleting a table

In [ ]:
# create_table(cursor, table_name, table_ddl)
def create_table(cursor, table_name, args):
    args_str = ','.join(args)
    query = "CREATE TABLE %s" % table_name + "(" + args_str + ")"
    exec_query(cursor, query)
    
# delete_table(cursor, table_name, table_ddl)
def delete_table(cursor, table_name):
    query = "DROP TABLE %s" % table_name
    exec_query(cursor, query)
    
if __name__ == '__main__':
    cnx = MySQLConnection(user='root', password='password')
    cursor = cnx.cursor()
    get_database(cnx, 'new_db')
    
    # New table
    name = "people"
    args = ("emp_no int(11) NOT NULL",
            "first_name varchar(14) NOT NULL",
            "last_name varchar(14) NOT NULL",
            "PRIMARY KEY (emp_no)"
           )
    create_table(cursor, name, args)
#     delete_table(cursor, name)

## Populating a table

In [ ]:
# add_employee(cursor, args)
def add_employee(cursor, db_name, emp_no, first_name, last_name):
    query = "INSERT INTO people(emp_no,first_name, last_name) " \
            "VALUES(%s,%s,%s)"
    args = (emp_no, first_name, last_name)
    print query % args
    exec_query(cursor, query, args)
    
if __name__ == '__main__':
    # Connect to database
    cnx = MySQLConnection(**config)
    cursor = cnx.cursor()

    # Get database
    get_database(cnx, 'new_db')

    # Insert new employee
    values = (10, "Geert", "Vanderkelen")
    add_employee(cursor, 'employee', *values)

    # Make sure data is committed to the database
    cnx.commit()
    cnx.close()

In [ ]:
# add_employees(cursor, employees)
def add_employees(cursor, employees):
    query = "INSERT INTO people(emp_no,first_name, last_name) " \
            "VALUES(%s,%s,%s)"
    for e in employees:
        exec_query(cursor, query, e)
    
if __name__ == '__main__':
    # Connect to database
    cnx = MySQLConnection(**config)
    cursor = cnx.cursor()

    # Get database
    get_database(cnx, 'new_db')

    # Insert new employee
    new_employees = [(10, "Geert", "Vanderkelen"), (11, "Olivier", "Cervello"), (12, "Gertrude", "Roger")]
    add_employees(cursor, new_employees)
    # delete_database(cursor, 'new_db')
                  
    # Make sure data is committed to the database
    cnx.commit()
    cnx.close()

## OOP Approach

In [ ]:
# OOP approach
class Controller(object):
    def __init__(self, cnx):
        self.cnx = cnx
        self.cursor = self.cnx.cursor()
        self.tables = []
        
    def get_db(self, name):
        # Get an existing database named db_name. 
        # If it doesn't exist, create a new database db_name.
        # Also get all tables in database.
        cursor = cnx.cursor()
        print "[INFO]  Fetching database %s ..." % name
        try:
            cnx.database = name    
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_BAD_DB_ERROR:
                print "[ERROR] Database %s does not exist! Creating database..." % name
                self.create_db(name)
                cnx.database = name
            else:
                print(err)
                exit(1)
        print "[INFO]  Database %s selected." % name

    def create_db(self, name):
        # Create a new database on the MySQL server
        query = "CREATE DATABASE %s" % name
        self.exec_query(query)
        
    def delete_db(self, name):
        # Delete an existing database.
        query = "DROP DATABASE %s" % name
        self.exec_query(query)
    
    def delete_table(self, name):
        # Delete a table from the database
        query = "DROP TABLE %s" % name
        self.exec_query(query)
            
    def create_table(self, name, args):
        # Create a new table in the database.
        args_str = ','.join(args)
        query = "CREATE TABLE IF NOT EXISTS %s" % name + "(" + args_str + ")"
        self.exec_query(query)
    
    def describe_table(self, name):
        query = "DESCRIBE %s" % name
        self.exec_query(query)
        try:
            import pandas
            df = pandas.DataFrame(self.cursor.fetchall())
            print df
            print
        except ImportError:
            print self.cursor.fetchall()
            print
        
    def exec_query(self, query, query_args=None):
        try:
            if query_args is not None:
                print "[INFO]  Executing SQL: \"%s\"..." % (query % query_args),
                self.cursor.execute(query, query_args)
            else:
                print "[INFO]  Executing SQL: \"%s\"..." % query,
                self.cursor.execute(query)
            print " SUCCESS"
        except mysql.connector.Error as err:
            print
            print "[ERROR] " + err.msg
            exit(1)
        
if __name__ == '__main__':
    cnx = mysql.connector.connect(user='root', password='password')
    controller = Controller(cnx)
    controller.get_db('new_db')
    controller.create_table('new_table', ["emp_no int(11) NOT NULL,first_name varchar(14) NOT NULL,last_name varchar(14) NOT NULL,PRIMARY KEY (emp_no)"])
    controller.describe_table('new_table')
    controller.exec_query("INSERT INTO new_table(emp_no, first_name, last_name) VALUES(%s, %s, %s)", query_args=(5, 'Olivier', 'Cervello'))
    controller.delete_table('new_table')
    controller.delete_db('new_db')

# NoSQL Data Access (DynamoDB)
**Amazon DynamoDB is a fast and flexible NoSQL database service for all applications that need consistent, single-digit millisecond latency at any scale.**

* **Module required**: ```boto3```
* **Install**: ```pip install boto3``` - https://pypi.python.org/pypi/boto3/

In [1]:
import boto3

### Client
**Description:** A low-level client representing Amazon DynamoDB.
** Methods available **

** Tables **
* ```list_tables()```
* ```create_table()```
* ```update_table```
* ```delete_table()```
* ```describe_table()```

** Items **
* ```get_item()```
* ```batch_get_item()```
* ```put_item()```
* ```batch_write_item()```
* ```delete_item()```

** Queries **
* ```query()```
* ```scan()```

### Resource
**Description:** A resource representing Amazon DynamoDB.
**Methods available**
* ```batch_get_item()```
* ```batch_write_item()```
* ```create_table()```

** Sub-resources **
* ```Table()``` - a Table resource

** Collections **
* ```tables``` - a collection of Table resources

### ```Table```
**Description:**
A ```Table``` resource.

**Parameters**
* ```name(string)``` - The table's name identifier. This **must** be set.

**Attributes**
* ```attribute_definitions```
* ```creation_date_time```
* ```global_secondary_indexes```
* ```item_count```
* ```key_schema```
* ```latest_stream_arn```
* ```local_secondary_indexes```
* ```provisioned_throughput```
* ```stream_specification```
* ```table_arn```
* ```table_name```
* ```table_size_bytes```
* ```table_status```

**Methods available**
* ```get_item()```
* ```put_item()```
* ```batch_writer()```
* ```delete()```
* ```delete_item()```
* ```query()```
* ```reload()```
* ```scan()```
* ```update()```
* ```update_item()```

### ```tables```
**Description:** A collection of ```Table``` resources.

**Methods**
* ```all()``` - iterable of Table resources in the collection
* ```filter(**kwargs)``` - iterable of Table resources in the collection **filtered by ```kwargs```**
* ```limit(**kwargs)``` - iterable **up to a specified amount** of Table resources in the collection
* ```page_size(**kwargs)``` - iterable of all Table resources in the collection, but **limits the number of items returned** by each service call by the specified amount.

## Connecting to DynamoDB

In [2]:
dynamodb_client = boto3.client('dynamodb', endpoint_url="http://localhost:9090")
dynamodb_client

In [3]:
dynamodb_resource = boto3.resource('dynamodb', endpoint_url="http://localhost:9090")
dynamodb_resource

dynamodb.ServiceResource()

## Creating / Deleting a table

**Client**

In [17]:
from botocore.exceptions import ClientError
import json

try:
    movies = dynamodb_client.create_table(
        TableName='Movies',
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'  #Partition key
            },
            {
                'AttributeName': 'title',
                'KeyType': 'RANGE'  #Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    
except ClientError as e:
    print "Table already exists."

# print dynamodb_client.list_tables()
movies

Table already exists.


{'ResponseMetadata': {'HTTPStatusCode': 200,
  'RequestId': '08afda95-a9c0-4edb-b185-4d9d8f563547'},
 u'TableDescription': {u'AttributeDefinitions': [{u'AttributeName': u'year',
    u'AttributeType': u'N'},
   {u'AttributeName': u'title', u'AttributeType': u'S'}],
  u'CreationDateTime': datetime.datetime(2015, 12, 16, 15, 13, 18, 484000, tzinfo=tzlocal()),
  u'ItemCount': 0,
  u'KeySchema': [{u'AttributeName': u'year', u'KeyType': u'HASH'},
   {u'AttributeName': u'title', u'KeyType': u'RANGE'}],
  u'ProvisionedThroughput': {u'LastDecreaseDateTime': datetime.datetime(1969, 12, 31, 18, 0, tzinfo=tzlocal()),
   u'LastIncreaseDateTime': datetime.datetime(1969, 12, 31, 18, 0, tzinfo=tzlocal()),
   u'NumberOfDecreasesToday': 0,
   u'ReadCapacityUnits': 10,
   u'WriteCapacityUnits': 10},
  u'TableArn': u'arn:aws:dynamodb:ddblocal:000000000000:table/Movies',
  u'TableName': u'Movies',
  u'TableSizeBytes': 0,
  u'TableStatus': u'ACTIVE'}}

**Resource**

In [23]:
from botocore.exceptions import ClientError

try:
    table = dynamodb_resource.create_table(
        TableName='Movies',
        KeySchema=[
            {
                'AttributeName': 'year',
                'KeyType': 'HASH'  #Partition key
            },
            {
                'AttributeName': 'title',
            'KeyType': 'RANGE'  #Sort key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
except ClientError as e:
    print "Table already exists."

movies = dynamodb_resource.Table('Movies')
print movies

dynamodb.Table(name='Movies')


## Populating a table

**Load JSON Data**

In [ ]:
%load files/moviedata.json

In [26]:
import json
import decimal

with open("files/moviedata.json") as json_file:
    movies = json.load(json_file, parse_float = decimal.Decimal)
    for movie in movies:
        year = movie['year']
        title = movie['title']
        info = movie['info']
        print "Adding movie:", year, title
        response = table.put_item(
            Item={
                'year': year,
                'title':title,
                'info': info,
            }
        )
        print response

Adding movie: 2000 The Skulls
{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': '6d3fa35f-dac5-41e5-ac00-1dfc2d7bdfc7'}}
Adding movie: 2012 Asterix et Obelix: Au service de Sa Majeste
{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': '494fa66e-eb5f-485e-9573-fda9b48a831d'}}
Adding movie: 2010 Lottery Ticket
{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': 'c420f008-86da-4e93-ac79-b0c8a8eb6764'}}
Adding movie: 1997 An American Werewolf in Paris
{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': 'fa0ec21d-3f19-4f27-8c9c-778e8faf48d1'}}
Adding movie: 1955 The Night of the Hunter
{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': '3d27ef95-da21-43c4-9960-4e4d9d982334'}}
Adding movie: 1994 In the Mouth of Madness
{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': 'bfe4880a-c012-4856-9f27-b844f8b16094'}}
Adding movie: 2011 Wrong Turn 4: Bloody Beginnings
{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': '8870c5ec-c187-4701-b051-970596d07a

**Add item**

In [ ]:
import json
import decimal

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

dynamodb = boto3.resource('dynamodb',endpoint_url="http://localhost:9090")

table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

response = table.put_item( # will overwrite any item with the same keys.
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':'Something happens.'
        }
    }
)

print "PutItem succeeded:"
print json.dumps(response, indent=4, cls=DecimalEncoder)

**Add item with condition**

In [ ]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

try:
    table.put_item(
        Item={
            'year': year,
            'title': title,
            'info':{
                'plot':"Nothing happens at all.",
                'rating': decimal.Decimal(0)
            }
        },
        ConditionExpression=Attr("year").ne(year) & Attr("title").ne(title)
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print e.response['Error']['Message']
    else:
        raise
else:
    print "PutItem succeeded:"

response = table.get_item(
    Key={
        'year': year,
        'title': title
    }
)
item = response['Item']

**Update an item**

In [ ]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="set info.rating = :r, info.plot=:p, info.actors=:a",
    ExpressionAttributeValues={
        ':r': decimal.Decimal(5.5),
        ':p': "Everything happens all at once.",
        ':a': ["Larry", "Moe", "Curly"]
    },
    ReturnValues="UPDATED_NEW"
)

print "PutItem succeeded:"
print json.dumps(response, indent=4, cls=DecimalEncoder)

**Update an item with condition**

In [ ]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

# Conditional update (will fail)
print "Attempting conditional update..."

try:
    response = table.update_item(
        Key={
            'year': year,
            'title': title
        },
        UpdateExpression="remove info.actors[0]",
        ConditionExpression="size(info.actors) >= :num",
        ExpressionAttributeValues={
            ':num': 3
        },
        ReturnValues="UPDATED_NEW"
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print e.response['Error']['Message']
    else:
        raise
else:
    print "PutItem succeeded:"

print(json.dumps(response, indent=4, cls=DecimalEncoder))

**Delete an item**

In [ ]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

table = dynamodb.Table('Movies')

title = "The Big New Movie"
year = 2015

print "Attempting a conditional delete..."

try:
    response = table.delete_item(
        Key={
            'year': year,
            'title': title
        },
        ConditionExpression="info.rating <= :val",
        ExpressionAttributeValues= {
            ":val": decimal.Decimal(5)
        }
    )
except ClientError as e:
    if e.response['Error']['Code'] == "ConditionalCheckFailedException":
        print e.response['Error']['Message']
    else:
        raise
else:
    print "PutItem succeeded:"

print json.dumps(response, indent=4, cls=DecimalEncoder)

## Querying data from a table

**Query: All movies released in a year**

In [ ]:
from boto3.dynamodb.conditions import Key, Attr

table = dynamodb.Table('Movies')

print("Movies from 1985")

response = table.query(
    KeyConditionExpression=Key('year').eq(1985)
)

for i in response['Items']:
    print i['year'], ":", i['title']

**Query: All movies released in a year with certain titles**

In [ ]:
from boto3.dynamodb.conditions import Key, Attr

table = dynamodb.Table('Movies')

print "Movies from 1992 - titles A-L, with genres and lead actor"

response = table.query(
    ProjectionExpression="#yr, title, info.genres, info.actors[0]",
    ExpressionAttributeNames={ "#yr": "year" }, # Expression Attribute Names for Projection Expression only.
    KeyConditionExpression=Key('year').eq(1985) & Key('title').between('A', 'L')
)

for i in response[u'Items']:
    print json.dumps(i, cls=DecimalEncoder)

**Scan**

In [ ]:
from boto3.dynamodb.conditions import Key, Attr

table = dynamodb.Table('Movies')

fe = Key('year').between(1950, 1959);
pe = "#yr, title, info.rating"
# Expression Attribute Names for Projection Expression only.
ean = { "#yr": "year", }
esk = None


response = table.scan(
    FilterExpression=fe,
    ProjectionExpression=pe,
    ExpressionAttributeNames=ean
    )

for i in response['Items']:
    print json.dumps(i, cls=DecimalEncoder)

while 'LastEvaluatedKey' in response:
    response = table.scan(
        ProjectionExpression=pe,
        FilterExpression=fe,
        ExpressionAttributeNames= ean,
        ExclusiveStartKey=response['LastEvaluatedKey']
        )

    for i in response['Items']:
        print json.dumps(i, cls=DecimalEncoder)

# Exercise: Twitter REST API
### Problem
**Objectives:** 
- Connect to Twitter API
- Extract interesting data

### Solution
* You can find the code for the Twitter REST API at the following address:
[https://github.com/ramusus/django-twitter-api](https://github.com/ramusus/django-twitter-api)
* This is actually also a ```pip``` package that you can install by running ```pip install django-twitter-api```. Refer to the Github repo's ```README.md``` to find out how to use it.